# Description

The goal of this notebook is to visualize and compare the optimization under 3 different schemes:

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import hydra

from omegaconf import OmegaConf, DictConfig
from pprint import pprint
from pathlib import Path

In [30]:
sns.set_style('ticks')
sns.set_context("paper", font_scale=1.0, rc={"text.usetex": True})
sns.set_palette("colorblind")
sns.set_style({"font.family": "serif", "font.serif": ["Times New Roman"]})
project_dir = Path('/n/home13/shubham/Current Projects/bioplausible_learning/code')

In [31]:
with hydra.initialize(version_base=None, config_path="../../../configs"):
    cfg = hydra.compose(config_name='ksm_high_dim.yaml')
    exp_params = OmegaConf.to_container(cfg, resolve=True)
    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

exp_params['device'] = 'device'

# figfolder = Path(f'../../results/simulated/highdim/{datetime.now().strftime("%m-%d")}/{datetime.now().strftime("%H-%M")}')
# os.makedirs(figfolder, exist_ok=True)    
pprint(exp_params)    

{'dataset': {'batch_size': 128,
             'dim': 30,
             'n_samples': 1500,
             'name': 'highdim'},
 'device': 'device',
 'model': {'K': 16,
           'Winit': 'Kmeans',
           'lam': 0.5,
           'omega': 0.0001,
           'perturbation': 0.1,
           'rho': 1.0,
           'sparsity': 0.125},
 'optimizer': {'Minit': 'zero',
               'latent_iters': 15,
               'log_interval': 10,
               'lrs': {'Z': 0.01,
                       'Z_decay': 0.9,
                       'interval': 10,
                       'params': 0.01},
               'max_epochs': 3000,
               'param_iters': 1,
               'vis_interval': 20}}


In [32]:
# Load loss curves
method = 'altmin'
result_folder = Path.cwd() / 'results' / '06-17' / f'{method}'
filename = f"ksm_{method}_K{exp_params['model']['K']}_lam{exp_params['model']['lam']:.2e}_n{exp_params['dataset']['dim']}_epochs3500.pkl"
with open(result_folder / filename, 'rb') as f:
    results_altmin = pickle.load(f)

In [33]:
loss_vals_altmin = results_altmin['loss_vals']
loss_vals_altmin.keys()

dict_keys(['total_loss', 'similarity_loss', 'l1_loss', 'penalty', 'sparsity', 'r2_W', 'r2_est', 'dw_sim', 'dest_sim', 'latent_sim'])

In [34]:
method = 'Ptrack'
result_folder = Path.cwd() / 'results' / '06-17' / f'{method}'
filename = f"ksm_{method}_K{exp_params['model']['K']}_lam{exp_params['model']['lam']:.2e}_n{exp_params['dataset']['dim']}_epochs3500.pkl"
with open(result_folder / filename, 'rb') as f:
    results_Ptrack = pickle.load(f)
loss_vals_Ptrack = results_Ptrack['loss_vals']

In [35]:
method = 'Notrack'
result_folder = Path.cwd() / 'results' / '06-17' / f'{method}'
filename = f"ksm_K{exp_params['model']['K']}_lam{exp_params['model']['lam']:.2e}_n{exp_params['dataset']['dim']}_epochs3500.pkl"
with open(result_folder / filename, 'rb') as f:
    results_Notrack = pickle.load(f)
loss_vals_Notrack = results_Notrack['loss_vals']

In [ ]:
# Let's plot the losses
fig, axs = plt.subplots(1, 4, figsize=(20, 4))
exp_params['optimizer']['log_interval'] = 10
exp_params['optimizer']['max_epochs'] = 3500
iterations = np.arange(0, exp_params['optimizer']['max_epochs'], step=exp_params['optimizer']['log_interval'])
ax = axs[0]
epochs_altmin = 3500 // exp_params['optimizer']['log_interval']
# epochs_altmin = iterations
ax.plot(iterations[1:epochs_altmin], loss_vals_altmin['total_loss'][1:], c='g', label='Schedule 1', linestyle='--')
ax.plot(iterations[1:epochs_altmin], loss_vals_Ptrack['total_loss'][1:], c='r', label='Schedule 2', linestyle=':', linewidth=3.0)
ax.plot(iterations[1:], loss_vals_Notrack['total_loss'][1:], c='b', label='Schedule 3', linestyle='-.')
ax.legend()
ax.set_title('Total Loss')

ax.set_xscale('log')
ax.set_xlabel('Iterations')
ax.set_ylabel('Loss')

ax = axs[1]
ax.plot(iterations[1:epochs_altmin], loss_vals_altmin['similarity_loss'][1:], c='b', label='Schedule 1')
ax.plot(iterations[1:epochs_altmin], loss_vals_Ptrack['similarity_loss'][1:], c='orange', label='Schedule 2')
ax.plot(iterations[1:], loss_vals_Notrack['similarity_loss'][1:], c='g', label='Schedule 3')
ax.legend()
ax.set_title('Similarity Loss')
ax.set_xscale('log')
ax.set_xlabel('Iterations')

ax = axs[2]
ax.plot(iterations[:epochs_altmin], loss_vals_altmin['sparsity'], c='g', label='Schedule 1', linestyle='--')
ax.plot(iterations[:epochs_altmin], loss_vals_Ptrack['sparsity'], c='r', label='Schedule 2', linestyle=':', linewidth=3.0)
ax.plot(iterations, loss_vals_Notrack['sparsity'], c='b', label='Schedule 3', linestyle='-.')
ax.set_title(r'Sparsity Measure $\lambda=0.5$')
ax.legend()
ax.set_xscale('log')
ax.set_xlabel('Iterations')

ax = axs[3]
ax.plot(iterations[:epochs_altmin], loss_vals_altmin['penalty'], c='b', label='Schedule 1')
ax.plot(iterations[:epochs_altmin], loss_vals_Ptrack['penalty'], c='orange', label='Schedule 2')
ax.plot(iterations, loss_vals_Notrack['penalty'], c='g', label='Schedule 3')
ax.set_title('Penalty')
ax.legend()
ax.set_xscale('log')
ax.set_xlabel('Iterations')


fig.savefig('train_losses.pdf', format='pdf', bbox_inches='tight')


# # ax.plot(loss_vals['loss_Z'])


# Plot the r2 score and similarity measures
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
ax = axs[0]
# ax.plot(iterations[1:epochs_altmin], loss_vals_altmin['r2_W'][1:], label='KSM altmin')
# ax.plot(iterations[1:epochs_altmin], loss_vals_Ptrack['r2_W'][1:], label='KSM P track')
# ax.plot(iterations[1:], loss_vals_Notrack['r2_W'][1:], label='KSM')
# ax.legend()
ax.plot(iterations[1:epochs_altmin], loss_vals_altmin['r2_est'][1:], label='Schedule 1', c='g', linestyle='--')
ax.plot(iterations[1:epochs_altmin], loss_vals_Ptrack['r2_est'][1:], label='Schedule 2', c='r', linestyle=':', linewidth=3.0)
ax.plot(iterations[1:], loss_vals_Notrack['r2_est'][1:], label='Schedule 3', c='b', linestyle='-.')
ax.legend()
# ax.plot(iterations[1:], loss_vals['r2_est'][1:], label=r'$R^2$ (Estimated)')
ax.set_xscale('log')
ax.set_title(r'$R^2$ values')
ax.set_xlabel('Iterations')
ax.set_ylabel('Value')
# ax.legend()

ax = axs[1]
ax.plot(iterations[:epochs_altmin], loss_vals_altmin['dw_sim'], label='Schedule 1')
ax.plot(iterations[:epochs_altmin], loss_vals_Ptrack['dw_sim'], label='Schedule 2')
ax.plot(iterations, loss_vals_Notrack['dw_sim'], label='Schedule 3')
# ax.plot(iterations, loss_vals['dest_sim'], label='cos sim (est)')
ax.set_xscale('log')
ax.set_title('Cos Similarity for estimated dictionary')
ax.set_xlabel('Iterations')
ax.set_ylabel('Value')
ax.legend()

ax = axs[2]
ax.plot(iterations[:epochs_altmin], loss_vals_altmin['latent_sim'], label='Schedule 1')
ax.plot(iterations[:epochs_altmin], loss_vals_Ptrack['latent_sim'], label='Schedule 2')
ax.plot(iterations, loss_vals_Notrack['latent_sim'], label='Schedule 3')
ax.set_xscale('log')
ax.set_title('Cos Similarity for latent codes')
ax.set_xlabel('Iterations')
ax.set_ylabel('Value')

# # fig.savefig(results_folder / 'r2_n_cossim.pdf', format='pdf', bbox_inches='tight')
fig.savefig('loss_curves.pdf', format='pdf', bbox_inches='tight')
fig.show()


In [ ]:
# compute likelihood ratio for r2 values
r2_W = np.array(loss_vals_altmin['r2_W'])
r2_est = np.array(loss_vals_altmin['r2_est'])
r2_W_notrack = np.array(loss_vals_Notrack['r2_W'])
r2_W_Ptrack = np.array(loss_vals_Ptrack['r2_W'])
r2_est_notrack = np.array(loss_vals_Notrack['r2_est'])
r2_est_Ptrack = np.array(loss_vals_Ptrack['r2_est'])
# Plot convergence rate of r2 values
epsilon = 1e-10
ratio = np.abs(r2_est[1:] - 1 + epsilon) / np.abs(r2_est[:-1] - 1 + epsilon)
# ratio = np.abs(np.diff(r2_W[1:]) / np.diff(r2_W[:-1]))
ration_notrack = np.abs(r2_est_notrack[1:] - 1 + epsilon) / np.abs(r2_est_notrack[:-1] - 1 + epsilon)
ratio_Ptrack = np.abs(r2_est_Ptrack[1:] - 1 + epsilon) / np.abs(r2_est_Ptrack[:-1] - 1 + epsilon)
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
# ax.plot(iterations[:-1], np.log10(ratio), label='KSM altmin')

r_k = np.log10(np.abs(r2_est[1:] - 1 + epsilon))
r_k1 = np.log10(np.abs(r2_est[:-1] - 1 + epsilon))
r_k_notrack = np.log10(np.abs(r2_est_notrack[1:] - 1 + epsilon))
r_k1_notrack = np.log10(np.abs(r2_est_notrack[:-1] - 1 + epsilon))
r_k_Ptrack = np.log10(np.abs(r2_est_Ptrack[1:] - 1 + epsilon))
r_k1_Ptrack = np.log10(np.abs(r2_est_Ptrack[:-1] - 1 + epsilon))
ax.plot(r_k1_Ptrack, r_k_Ptrack, label='Schedule 2', c='r', linestyle=':', linewidth=3)
ax.plot(r_k1, r_k, label='Schedule 1', linestyle='--', c='g')
ax.plot(r_k1_notrack, r_k_notrack, label='Schedule 3', c='b', linestyle='-.')
# ax.plot(iterations[1:], ration_notrack, label='KSM')
# ax.plot(iterations[1:], ratio_Ptrack, label='KSM P track')
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.xaxis.set_label_position('top') 
ax.yaxis.tick_right()
ax.set_ylabel(r'$\log_{10}(|R^2_{k} - 1|)$')

ax.set_xlabel(r'$\log_{10}(|R^2_{k-1} - 1|)$')
ax.legend()
ax.set_title('Convergence rate of $R^2$ values', y=-0.05)
fig.savefig('r2_convergence.pdf', format='pdf', bbox_inches='tight')
# ax.set_xscale('log')
# ax.set_yscale('log')